In [1]:
import mne
import  numpy as np
import os
import matplotlib.pyplot as plt
from mne.preprocessing import ICA

import warnings

# Ignore all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:



eog = ['AF1','AF2']


misc = ['thumb_near',
 'thumb_far',
 'thumb_index',
 'index_near',
 'index_far',
 'index_middle',
 'middle_near',
 'middle_far',
 'middle_ring',
 'ring_near',
 'ring_far',
 'ring_little',
 'litte_near',
 'litte_far',
 'thumb_palm',
 'wrist_bend',
 'roll',
 'pitch',
 'gesture',
 'armeodummy',
 'handPosX',
 'handPosY',
 'handPosZ',
 'elbowPosX',
 'elbowPosY',
 'elbowPosZ',
 'ShoulderAdductio',
 'ShoulderFlexionE',
 'ShoulderRotation',
 'Elbow',
 'ProSupination',
 'Wrist',
 'GripPressure'
 ]
scalings = dict(mag=1e-12, grad=4e-11, eeg=10e-5, eog=150e-6, ecg=5e-4,
     emg=1e-3, ref_meg=1e-12, misc=1e-3, stim=1,
     resp=1, chpi=1e-4, whitened=1e2)

In [3]:
base_path = "./Subject_dependent/"

In [4]:
def eeg_files_find(subject_id,r_or_img_id,path_subject = base_path):
    idx = os.listdir(path_subjects)[subject_id]
    r_or_img = ["Real","Imagery"][r_or_img_id]
    print(idx,r_or_img)
    path_2 = path_subjects+"/"+idx+"/"+r_or_img
    d =  os.listdir(path_2)
    eeg_file = []
    for i in range(len(d)):
      if d[i][-4:]=="vhdr":
#         print(d[i] , end ="\n")
        eeg_file.append(d[i])
    return eeg_file,path_2+"/",idx

In [5]:

def eeg_raw(path_2,eeg_file):
       
    raws_2 = []
    raws = [mne.io.read_raw_brainvision(path_2 +f,eog=eog, preload = True) for f in eeg_file]
    try:
        for i in range(len(raws)):
            Temp= list(set(raws[i].pick(["eeg"]).info["ch_names"])-set(["F3","C3","P3","F4","C4","P4","Fz","Cz","Pz","TP9"]))
            raws_2.append(raws[i].pick(["eeg"]).drop_channels(Temp))
#             print(raws[i].pick(["eeg"]).info["ch_names"])

        raws_2 = mne.io.concatenate_raws(raws_2)
        raws_2.set_eeg_reference(ref_channels=['TP9'])
        raws_2.drop_channels(['TP9'])
    except:
        raws_2 = []
        for i in range(len(raws)):
            Temp= list(set(raws[i].pick(["eeg"]).info["ch_names"])-set(["F3","C3","P3","F4","C4","P4","Fz","Cz","Pz"]))
            raws_2.append(raws[i].pick(["eeg"]).drop_channels(Temp))
#             print(raws[i].pick(["eeg"]).info["ch_names"])

        raws_2 = mne.io.concatenate_raws(raws_2)
        
        
    
    return raws_2


In [6]:
def event_name_id(raws):
    events, event_dict = mne.events_from_annotations(raws)
    print(event_dict,events)
    annotations_dict = {k:k[8:] if k[:8] == 'Comment/' else k for k in event_dict.keys()}
    print(annotations_dict)
    event_id = {"Close":event_dict["Comment/Close"],
        "Open":event_dict["Comment/Open"],
        "Rest":event_dict["Comment/Rest"]}
    print(event_id)
    
    return events,event_id

In [7]:
def epoching(raw,events,event_id):
    tmin =  0 # start of each epoch (in sec)
    tmax =  3  # end of each epoch (in sec)
    # baseline = (None, 0)
    # baseline =  (0.1,0.2)
    baseline = None #(-1,0)
    
    
    
#     sample_freq = 250
#     raw.resample(sample_freq)
    
    
    
    # Create epochs
    epochs = mne.Epochs(raw,
                        events, event_id,
                        tmin, tmax,
                        baseline=baseline,
                        preload=True
                       )
    return epochs

In [8]:
def event_data(epoch):
    
    event_len = len(epoch["Open"].get_data())
    Close_data = epoch["Close"][:event_len].get_data()
    Open_data = epoch["Open"].get_data()
    Rest_data = epoch["Rest"][:event_len].get_data()
    
    print(Close_data.shape , Open_data.shape , Rest_data.shape)
    return Close_data, Open_data, Rest_data

In [14]:
def data_file_save(path_2,eeg_file,subject):
    x = eeg_raw(path_2,eeg_file)
    y, z = event_name_id(x)
    a = epoching(x,y,z)
    c, o, r = event_data(a)
    label_c =  np.zeros(len(c))+1
    label_o = np.zeros(len(c))+2
    label_r = np.zeros(len(r))+3
    final_label = np.concatenate((label_c,label_o,label_r ))
    np.save(f"./Data_file/label_{subject}.npy",final_label,)
    data = np.concatenate((c, o, r))
    np.save(f"./Data_file/data_{subject}.npy", data)

In [11]:
path_subjects=base_path
for i,j in enumerate(os.listdir(path_subjects)):
    print(i+1,j)


1 Abha
2 Aman
3 Anuj Sir
4 Aryan
5 Dipti
6 Harsha
7 jay
8 Mayank
9 Mintu
10 Moumita
11 Mudavath
12 Prashant
13 Ritesh
14 Sharon
15 Shivam
16 Soinik
17 Suma
18 Zeeshan


In [12]:
os.makedirs(os.path.join("./", "Data_file"), exist_ok=True)

In [18]:
%%capture
for i in range(18):
    eeg_file,path_2,name_of_subject = eeg_files_find(i,1)
    data_file_save(path_2,eeg_file, name_of_subject)